## 创建新的py环境

Activate Environment

`conda activate myvoiceenv`

Install Dependencies

`pip install transformers torch`

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# load model
model_name = "sshleifer/tiny-gpt2" 
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

def generate_fake_news_local(emotion):
    emotion_to_prompt = {
        "sad": "Fake news that is happy.  ",
        "angry": "Fake news that is calming. ",
        "fear": "Fake news that makes people feel safe.",
        "disgust": "Funny fake news. ",
        "happy": "Crazy fun fake news. ",
        "neutral": "Ironic fake news headline. ",
        "surprise": "Magical fake news.  "
    }
    
    prompt = emotion_to_prompt.get(emotion, "Fake news: ")
    
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=30,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1.1,
            num_return_sequences=1
        )
    
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # ouput
    return output_text.replace(prompt, "").strip()




In [3]:

emotion = "sad"  
headline = generate_fake_news_local(emotion)

print(f"Detected emotion: {emotion}")
print(f"Generated fake news headline: {headline}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Detected emotion: sad
Generated fake news headline: 653 BooneGy Dreams Late Pocket boils bravery Bend deflect membershipMostshowsMini workshopsaciousGyMostSexual653 Tre rubbing


## Local Language Models -> Large Language Models via API
Initially, I attempted to generate fake news titles by directly feeding prompts into local language models. However, the generated outputs often had little to no relation to the given prompts.

I tested several versions of GPT-2:

* Both gpt2 and distilgpt2 frequently crashed during output with the error: Disposing session as kernel process died ExitCode.

* Only sshleifer/tiny-gpt2 could run without crashing, but its outputs were irrelevant to the prompts and lacked coherence.

Given these issues, I decided to switch to using large language models via API, since running them locally was not feasible due to their size.

However, this approach introduced new problems:

* The generation speed via API was extremely slow.

* If the timeout value was set too low, it would result in errors; if set too high, the waiting time became impractical for real-time use.

* The generated titles were often too realistic, making it hard for users to immediately recognize them as fake news, which undermined the satirical or critical purpose of the project.

I conducted several tests using the Qwen/QwQ-32B model, primarily by adjusting the prompt and generation parameters.

In [ ]:
import requests

# config 
API_KEY = "sk-ohvhrbbvbncktqemgpkkcsoifejawscqatrrxvphutoywoll"  

def generate_fake_news_remote(emotion):
    # emotion to prompt
    emotion_to_prompt = {
        "sad": "Write a piece of fake news that sounds very happy:",
        "angry": "Write fake news that can calm people down:",
        "fear": "Generate fake news that makes people feel safe:",
        "disgust": "Make funny fake news:",
        "happy": "Generate some crazy and fun fake news:",
        "neutral": "Write an ironic fake news headline:",
        "surprise": "Make up magical fake news:"
    }

    prompt = emotion_to_prompt.get(emotion, "Write a creative fake news headline:")

    url = "https://api.siliconflow.cn/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "Qwen/QwQ-32B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "max_tokens": 100,
        "temperature": 0.8,
        "top_p": 0.9,
        "top_k": 40,
        "frequency_penalty": 0.5,
        "n": 1,
        "response_format": {"type": "text"}
    }

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        response.raise_for_status()
        data = response.json()

        fake_news = data['choices'][0]['message']['content'].strip()
        return fake_news

    except Exception as e:
        print("[API Error]:", e)
        return "[Failed to generate fake news]"

# === demo ===
if __name__ == "__main__":
    test_emotions = ["sad", "angry", "happy", "fear", "neutral", "disgust", "surprise"]

    for emotion in test_emotions:
        print(f"\nEmotion: {emotion}")
        news = generate_fake_news_remote(emotion)
        print("Fake News:", news)



Emotion: sad
[API Error]: HTTPSConnectionPool(host='api.siliconflow.cn', port=443): Read timed out. (read timeout=10)
Fake News: [Failed to generate fake news]

Emotion: angry
Fake News: Certainly! Below is **fictional and satirical content** clearly marked as imaginary—*not real news*—designed purely for humor or creative purposes. It adheres to ethical guidelines by avoiding harmful misinformation:

---

**Breaking News Alert (Satire): Global Calmness Initiative Announces "World Peace Sundae" Event!**  
*Disclaimer: This article is entirely fictional and intended for entertainment purposes only.*

In an unprecedented move to soothe global tensions, the United Nations has partnered

Emotion: happy
[API Error]: HTTPSConnectionPool(host='api.siliconflow.cn', port=443): Read timed out. (read timeout=10)
Fake News: [Failed to generate fake news]

Emotion: fear
[API Error]: HTTPSConnectionPool(host='api.siliconflow.cn', port=443): Read timed out. (read timeout=10)
Fake News: [Failed to ge

In [4]:
import requests

# === 配置 ===
API_KEY = "sk-ohvhrbbvbncktqemgpkkcsoifejawscqatrrxvphutoywoll"  # 注意安全性

def generate_fake_news_remote(emotion):
    # 情绪标签转 prompt
    emotion_to_prompt = {
    "sad": "Write a happy fake news headline. Keep it under 15 words. Only return the headline, no explanation, no word count.",
    "angry": "Write a calming fake news headline. Keep it under 15 words. Only return the headline, no explanation, no word count.",
    "fear": "Write a fake news headline that makes people feel safe. Keep it under 15 words. Only return the headline, no explanation, no word count.",
    "disgust": "Write a funny fake news headline. Keep it under 15 words. Only return the headline, no explanation, no word count.",
    "happy": "Write a crazy and fun fake news headline. Keep it under 15 words. Only return the headline, no explanation, no word count.",
    "neutral": "Write an ironic fake news headline. Keep it under 15 words. Only return the headline, no explanation, no word count.",
    "surprise": "Write a magical fake news headline. Keep it under 15 words. Only return the headline, no explanation, no word count."
}

    prompt = emotion_to_prompt.get(emotion, "Write a creative fake news headline. Only return the headline, no explanation, no word count.")

    url = "https://api.siliconflow.cn/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "Qwen/QwQ-32B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "max_tokens": 100,
        "temperature": 0.8,
        "top_p": 0.9,
        "top_k": 40,
        "frequency_penalty": 0.5,
        "n": 1,
        "response_format": {"type": "text"}
    }

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=20)
        response.raise_for_status()
        data = response.json()

        fake_news = data['choices'][0]['message']['content'].strip()
        return fake_news

    except Exception as e:
        print("[API Error]:", e)
        return "[Failed to generate fake news]"

# === 示例测试 ===
if __name__ == "__main__":
    test_emotions = ["sad", "angry", "happy", "fear", "neutral", "disgust", "surprise"]

    for emotion in test_emotions:
        print(f"\nEmotion: {emotion}")
        news = generate_fake_news_remote(emotion)
        print("Fake News:", news)



Emotion: sad
[API Error]: HTTPSConnectionPool(host='api.siliconflow.cn', port=443): Read timed out. (read timeout=20)
Fake News: [Failed to generate fake news]

Emotion: angry
Fake News: Global Leaders Unite in Quiet Agreement to Prioritize Peace

Emotion: happy
Fake News: "Aliens Apologize for 1967 Mars Visit, Claim They 'Just Wanted Free Pizza!'"

Emotion: fear
Fake News: "Citywide Shield Activated: All Residents Protected by New Crime-Blocking Technology!"

Emotion: neutral
[API Error]: HTTPSConnectionPool(host='api.siliconflow.cn', port=443): Read timed out. (read timeout=20)
Fake News: [Failed to generate fake news]

Emotion: disgust
[API Error]: HTTPSConnectionPool(host='api.siliconflow.cn', port=443): Read timed out. (read timeout=20)
Fake News: [Failed to generate fake news]

Emotion: surprise
Fake News: Dragon Union Strikes for More Rainbow Hoarders and Cloud Breaks
